# Artificial Intelligence - Fall 2022 - Laboratory 08

## _First Order Predicate Logic:  Representation and Unification_


## Introduction

The goal of this laboratory is to become familiar with the logical representation and the working mechanism of knowledge within the knowledge represented by logic with first-order predicates (FOPL).

In the lab, you will need to choose an internal representation for the elements in the FOPL, and then implement the unification process between two formulas in predicate logic.

**_Useful resources:_**

* AI Course #5
* https://en.wikipedia.org/wiki/Unification_(computer_science)#Examples_of_syntactic_unification_of_first-order_terms
* Robinson algorithm

## Representation

In the **FOPL** we must be able to represent the following elements:

* _term_ - can be taken as an argument by a predicate. A term can be:
   * a constant - has a value
   * a variable - has a name and can be linked to a value
   * a function call - has the function name and arguments (e.g. `add[1, 2, 3]`). It is evaluated at a value. The arguments of the function are also terms.
   
     _Note:_ In the text we will write the function calls with square brackets, to distinguish them from atoms.


* _logical formula (sentence )_ - can be evaluated for a certain truth value, in an interpretation (a connection between names and semantics). A formula can be:
   * an atom - the application of a predicate (with a name) over a series of terms (its arguments)
   * negation of a formula
   * a logical connector between two sentences - conjunction or disjunction



Create an internal representation for logical formulas. For this representation, we will have:
* a series of functions that build it - `make_ *` and `replace_args`
* a series of functions that check it - `is_ *`
* a series of functions that access the elements in the representation - `get_ *`


**_Important:_** To work more easily with formulas, we will consider that for function calls and all formulas (both atoms and compound sentences), the representation has a `head` (as the case may be, the function name, predicate name, or logical connector) and a `list of arguments` (as appropriate, the argument list of the function, the argument list of the predicate, a list with the negative sentence, or the list of sentences joined by a logical connector (2 or more)).



In [1]:
# Useful libraries:
from operator import add
from LPTester import test_batch

In [2]:
### Representation - construction

# Returns a constant term with the specified value.
def make_const(value):
    return (value,)

# Returns a term that is a variable with the specified name.
def make_var(name):
    return (name, None)

# Returns a term that is a call to the specified function, on the rest of the given arguments.
# E.g. to build the term add [1, 2, 3] we will call -->
# --> make_function_call(add, make_const(1), make_const(2), make_const(3))
# !! WARNING: python gives args as a tuple with the rest of the arguments, not as a list. 
# The conversion can be realised using list(args)
def make_function_call(function, *args):
    return (function, list(args))

# Returns a formula consisting of an atom which is the utilisation of the given predicate
# on the rest of the additional arguments(*args).
# !! WARNING: python gives args as a tuple with the rest of the arguments, not as a list. 
# The conversion can be realised using list(args)
def make_atom(predicate, *args): 
    return (predicate, args)

# Returns a formula that is the negation of the given sentence.
# get_args(make_neg(s1)) will return [s1]
def make_neg(sentence):
    return {'~': [sentence]}

# Returns a formula that is the conjunction of the given sentences (2 or more).
# e.g. the call of the function make_and(s1, s2, s3, s4) returns the conjunction structure of s1 ^ s2 ^ s3 ^ s4
# and get_args for this structure returns [s1, s2, s3, s4]
def make_and(sentence1, sentence2, *others):
    return {'^': [sentence1, sentence2, *list(others)]}

# Returns a formula which is the disjunction of the given sentences.
# e.g. the call of the function make_or(s1, s2, s3, s4) returns the disjunction structure of s1 V s2 V s3 V s4
# and get_args for this structure returns [s1, s2, s3, s4]
def make_or(sentence1, sentence2, *others):
    return {'V': [sentence1, sentence2, *list(others)]}

# Returns a copy of the given formula or function call, 
# in which the arguments have been replaced with those in the new_args list.
# e.g. for formula p (x, y), replacing the arguments with list [1, 2] will result in formula p (1, 2).
# The new argument list must have the same length as the original number of arguments in the formula.
def replace_args(formula, new_args):
    if is_function_call(formula):
        return make_function_call(get_head(formula), *new_args)
    elif is_atom(formula):
        return make_atom(get_head(formula), *new_args)
    elif is_sentence(formula):
        if get_head(formula) == '~':
            return make_neg(*new_args)
        if get_head(formula) == 'V':
            return make_or(*new_args)    
        if get_head(formula) == '^':
                return make_and(*new_args)    
    else:
        return None

In [3]:
### Representation - verification

# Returns true if f is a term.
def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

# Returns true if f is a constant term.
def is_constant(f):
    if type(f) == tuple and len(f) == 1:
        return True
    else:
        return False

# Returns true if f is a term that is a variable.
def is_variable(f):
    if type(f) == tuple and len(f) == 2 and type(f[1]) != tuple and type(f[1]) != list:
        return True
    else:
        return False

# Returns true if f is a function call.
def is_function_call(f):
    if type(f) == tuple and len(f) == 2 and type(f[1]) == list:
        return True
    return False

# Returns true if f is an atom (application of a predicate).
def is_atom(f):
    if type(f) == tuple and len(f) == 2 and type(f[1]) == tuple:
        return True
    return False

# Returns true if f is a valid sentence.
def is_sentence(f):
    if is_atom(f) or type(f) == dict and list(f.keys())[0] in ['~','^','V']:
        return True
    return False

# Returns true if the formula f is something that has arguments..
def has_args(f):
    return is_function_call(f) or is_sentence(f)

# For constants (to be checked), the value of the constant is returned; otherwise, None.
def get_value(f):
    if is_constant(f):
        return f[0]
    else:    
        return None

# For variables (to be checked), return the name of the variable; otherwise, None.
def get_name(f):
    if is_variable(f):
        return f[0]
    else:
        return None

# for function calls, return the function;
# for atoms, the name of the predicate is returned;
# for compound sentences, return a string representing the logical connector (e.g. ~, A or V);
# otherwise, None
def get_head(f):
    if is_function_call(f):
        return f[0]
    if is_atom(f):
        return f[0]
    if is_sentence(f): 
        return list(f.keys())[0]
    else:
        return None

# For sentences or function calls, the list of arguments is returned; otherwise, None.
# See also "Important:" above.
def get_args(f):
    if is_function_call(f):
        return f[1]
    elif is_atom(f):
        return list(f[1])
    elif is_sentence(f):
        return list(f.values())[0]
    else:    
        return []

test_batch(0, globals())

>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
Test 24: OK
Test 25: OK
Test 26: OK
Test 27: OK
Test 28: OK
Test 29: OK
Test 30: OK
Test 31: OK
Test 32: OK
>>>  33 / 33 tests successful.


In [4]:
# This function displays the formula f. 
# If the return_result argument is True, the result is returnd and not displayed on the console.
def print_formula(f, return_result = False):
    ret = ""
    if is_term(f):
        if is_constant(f):
            ret += str(get_value(f))
        elif is_variable(f):
            ret += "?" + get_name(f)
        elif is_function_call(f):
            ret += str(get_head(f)) + "[" + "".join([print_formula(arg, True) + "," for arg in get_args(f)])[:-1] + "]"
        else:
            ret += "???"
    elif is_atom(f):
        ret += str(get_head(f)) + "(" + "".join([print_formula(arg, True) + ", " for arg in get_args(f)])[:-2] + ")"
    elif is_sentence(f):
        # negation, conjunction or disjunction
        args = get_args(f)
        if len(args) == 1:
            ret += str(get_head(f)) + print_formula(args[0], True)
        else:
            ret += "(" + str(get_head(f)) + "".join([" " + print_formula(arg, True) for arg in get_args(f)]) + ")"
    else:
        ret += "???"
    if return_result:
        return ret
    print(ret)
    return
    
# Verify construction and display
# The output should be similar to: (A (V ~P(?x) Q(?x)) T(?y, <built-in function add>[1,2]))
formula1 = make_and(
    make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))),
    make_atom("T", make_var("y"), make_function_call(add, make_const(1), make_const(2))))
print_formula(formula1)

(^ (V ~P(?x) Q(?x)) T(?y, <built-in function add>[1,2]))


## Unification

The unification of two logical formulas containing variables means finding a substitution so that its application on the two formulas results in two identical formulas.

A substitution contains associations of variables to terms. When a substitution is applied, the variables that appear in the substitution are replaced, in the formula, by the associated terms, until no replacement can be made.

We represent a substitution as a dictionary `{variable name: term representation}`

In [5]:
# This function applies in formula f all elements of the given substitution and returns the resulting formula
def substitute(f, substitution):
    if substitution is None:
        return None
    if is_variable(f) and (get_name(f) in substitution):
        return substitute(substitution[get_name(f)], substitution)
    if has_args(f):
        return replace_args(f, [substitute(arg, substitution) for arg in get_args(f)])
    return f

def test_formula(x, copyy = False):
    fun = make_function_call(add, make_const(1), make_const(2))
    return make_and(make_or(make_neg(make_atom("P", make_const(x))), make_atom("Q", make_const(x))), \
                    make_atom("T", fun if copyy else make_var("y"), fun))


# Test (the effects of substitutions in the formula must be seen)
#test_batch(1, globals())

### Task 1

Implement the 'unify` function, according to Robinson's algorithm (see pdf).

In [186]:
# Check if the variable v appears in the term t, considering the substitution of subst.
# Returns True if v appears in t (v can NOT be replaced with t), and False if v can be replaced with t.
def occur_check(v, t, subst):
    if v == t:
        return True
    elif get_name(t) in subst:
        return occur_check(v, substitute(t, subst), subst)
    elif is_function_call(t):
        return any([occur_check(v,a,subst) for a in get_args(t)])
    else:
        return False

# Test!
test_batch(2, globals())


>>> Test batch [2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
>>>  7 / 7 tests successful.


In [187]:
def unify(f1, f2, subst = None):
    if subst is None:
        subst = {}

    s=f1
    t=f2
    stack=[[s,t]]
    while len(stack)!=0:
        s,t=stack.pop()
        if is_variable(s) and s[0] in subst:
            s=substitute(s,subst)
        if is_variable(t) and t[0] in subst:
            t=substitute(t,subst)
        if s!=t:
            if is_variable(s):
                if occur_check(s,t,subst):
                    return False
                else:
                    subst[s[0]]=t
            elif is_variable(t):
                if occur_check(t,s,subst):
                    return False
                else:
                    subst[t[0]]=s
            if is_constant(s) and is_constant(t):
                return False
            if has_args(s) and has_args(t):
                if get_head(s)==get_head(t):
                    arg1=get_args(s)
                    arg2=get_args(t)
                    if len(arg1)==len(arg2):
                        for i in range(len(arg1)):
                            stack.append([arg1[i],arg2[i]])
                else:
                    return False


    return subst

test_batch(3, globals())

>>> Test batch [3]
Test 0: OK, got < False >
Test 1: OK, got < {?x -> A} >
Test 2: OK, got < {?x -> A} >
Test 3: OK, got < {?x -> A} >
Test 4: OK, got < {?x -> A} >
Test 5: OK, got < {?x -> <built-in function add>[?Z,5]} >
Test 6: OK, got < {?z -> C, ?y -> B, ?x -> A} >
Test 7: OK, got < {?y -> 3, ?x -> 2} >
Test 8: OK, got < False >
Test 9: OK, got < False >
Test 10: OK, got < {} >
Test 11: OK, got < {?x -> ?y} >
Test 12: OK, got < {?y -> ?x} >
Test 13: OK, got < {?x -> A} >
Test 14: OK, got < False >
Test 15: OK, got < {?x -> A} >
Test 16: OK, got < {?x -> A, ?z -> C, ?y -> B} >
Test 17: OK, got < {?x -> A} >
Test 18: OK, got < {?x -> ?y} >
Test 19: OK, got < {?y -> ?x} >
Test 20: OK, got < {?z -> 5, ?y -> 5, ?x -> 5} >
Test 21: OK, got < {?z -> <built-in function add>[1,2], ?y -> 4, ?x -> 3} >
>>>  22 / 22 tests successful.
